In [3]:
pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 1.5 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd

from sklearn.model_selection import GroupKFold, train_test_split 

from sklearn.exceptions import ConvergenceWarning
import warnings

# Class data_prep

In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)

class DataPreparation:
    def __init__(self, train, test, neural_networks = False, target = None):
        self.train = train.copy()
        self.test = test.copy()
        self.target = target
        self.neural_networks = neural_networks

    def get_variable_correlation(self, variable):
        correlation_vector = self.train[self.col_numericals].corr()[variable][:]
        correlation_vector = np.abs(correlation_vector)
        correlation_vector = correlation_vector.sort_values(ascending=False)[1:]

        return (correlation_vector)

    def get_nan_table(self):
        nan_table = pd.DataFrame(columns=["Variable", "Pourcentages de valeurs manquantes", "Type"])

        for col in self.train:
            pcentage = (self.train[col].isna().sum() / self.train.shape[0]) * 100
            type = 'Numérique' if col in self.col_numericals else 'Catégorielle'
            nan_table.loc[len(nan_table)] = [col, pcentage, type]

        nan_table = nan_table.sort_values(by=["Pourcentages de valeurs manquantes"], ascending=False).reset_index(
            drop=True)

        return nan_table

    def remove_train_nan(self):
        percentage = (self.train.isna().sum() / self.train.shape[0]) * 100
        self.columns_to_delete = [col for col in self.train.columns if
                                  col != "Electric range (km)" and percentage[col] >= 40]
        self.columns_to_delete.extend(["r", "Status", "IT", 'Date of registration', "Mp"])
        self.train.drop(columns=self.columns_to_delete, inplace=True)
        print("Valeurs manquantes du train supprimées ✅")

    def remove_test_nan(self):
        self.test.drop(columns=self.columns_to_delete, inplace=True)
        print("Valeurs manquantes du test supprimées ✅")

    def rename_columns(self):
        self.train.columns = self.train.columns.str.replace(' ', '_')
        self.test.columns = self.test.columns.str.replace(' ', '_')
        print("Variables renommées ✅")

    def get_type_list(self):
        self.col_categoricals = self.train.select_dtypes(include=['object']).columns.to_list()
        self.col_numericals = self.train.select_dtypes(exclude=["object"]).columns.to_list()
        self.col_numericals.remove("ID")

    def impute_by_mean(self, var_to_impute, var_mean):
        mean_train = self.train.groupby(var_mean)[var_to_impute].transform('mean')
        self.train[var_to_impute].fillna(mean_train, inplace=True)
        self.train[var_to_impute].fillna(self.train[var_to_impute].mean(), inplace=True)

        mean_test = self.test.groupby(var_mean)[var_to_impute].transform('mean')
        self.test[var_to_impute].fillna(mean_test, inplace=True)
        self.test[var_to_impute].fillna(self.test[var_to_impute].mean(), inplace=True)

    def impute_train_test_numerical(self):

        # Imputation of Electric range
        self.train['Electric_range_(km)'].fillna(0, inplace=True)
        self.test['Electric_range_(km)'].fillna(0, inplace=True)

        # Imputation of Ec
        self.train["ec_(cm3)"].fillna(0, inplace=True)
        self.test['ec_(cm3)'].fillna(0, inplace=True)

        #Imputation of Mt
        self.train.loc[self.train["Mt"].isna(), "Mt"] = self.train.loc[self.train["Mt"].isna()]["m_(kg)"]
        self.train["Mt"].fillna(self.train["Mt"].mean(), inplace=True)

        self.test.loc[self.test["Mt"].isna(), "Mt"] = self.test.loc[self.test["Mt"].isna()]["m_(kg)"]
        self.test["Mt"].fillna(self.test["Mt"].mean(), inplace=True)

        #Imputation of Fuel Consumption

        self.train.loc[(self.train["Fuel_consumption_"].isna()) & (self.train["Ft"] == 'ELECTRIC'), "Fuel_consumption_"] = 0
        self.test.loc[(self.test["Fuel_consumption_"].isna()) & (self.test["Ft"] == 'ELECTRIC'), "Fuel_consumption_"] = 0

        self.impute_by_mean("Fuel_consumption_", "Cn")

        #Imputation of At1 (mm)
        self.impute_by_mean("At1_(mm)", "Cn")

        # Imputation of At2 (mm)
        self.impute_by_mean("At2_(mm)", "Cn")

        #Imputation of m (kg)
        self.impute_by_mean("m_(kg)", "Cn")

        #Imputation of W
        self.impute_by_mean("W_(mm)", "Cn")

        #Imputation of ep
        self.impute_by_mean("ep_(KW)", "Cn")

        print("Valeurs manquantes numériques imputées ✅")

    def impute_train_test_categorical(self):
        #Imputation of 'Cn'
        mode_VFN_train = self.train.groupby('T')['Cn'].apply(
            lambda x: x.mode().iloc[0] if not x.mode().empty else None)
        self.train['Cn'] = self.train['Cn'].fillna(self.train['T'].map(mode_VFN_train))

        mode_VFN_test = self.test.groupby('T')['Cn'].apply(
            lambda x: x.mode().iloc[0] if not x.mode().empty else None)
        self.test['Cn'] = self.test['Cn'].fillna(self.test['T'].map(mode_VFN_test))

        # Impute 'Cn' par le mode de Cn si T est manquant
        self.train['Cn'].fillna(self.train['Cn'].mode()[0], inplace=True)

        self.test['Cn'].fillna(self.test['Cn'].mode()[0], inplace=True)


        # Imputation of 'VFN'
        mode_VFN_train = self.train.groupby('Cn')['VFN'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
        self.train['VFN'] = self.train['VFN'].fillna(self.train['Cn'].map(mode_VFN_train))

        mode_VFN_test = self.test.groupby('Cn')['VFN'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else None)
        self.test['VFN'] = self.test['VFN'].fillna(self.test['Cn'].map(mode_VFN_test))

        # Impute 'VFN' with 'Va' if 'Cn' is missing
        self.train['VFN'] = self.train.apply(
            lambda row: row['Va'] if pd.isna(row['VFN']) and not pd.isna(row['Cn']) else row['VFN'],
            axis=1)

        self.test['VFN'] = self.test.apply(
            lambda row: row['Va'] if pd.isna(row['VFN']) and not pd.isna(row['Cn']) else row['VFN'],
            axis=1)

        # Impute 'VFN' with mode of 'VFN' if both 'Cn' and 'Va' are missing
        self.train['VFN'].fillna(self.train['VFN'].mode()[0], inplace=True)

        self.test['VFN'].fillna(self.test['VFN'].mode()[0], inplace=True)

        # Imputation des variables ayant moins de 1% de NaN
        for col in ['Tan', 'T', 'Va', 'Ve', 'Mk', 'Ct', 'Fm']:
            self.train[col].fillna(self.train[col].mode()[0],inplace=True)
            self.test[col].fillna(self.train[col].mode()[0], inplace=True)

        self.train.Ft = self.train.Ft.apply(lambda x: "PETROL" if x == "UNKNOWN" else x)
        self.train["Ewltp_(g/km)"] = self.train["Ewltp_(g/km)"].apply(lambda x: 0 if x < 0 else x)
        self.train.Ft = self.train.Ft.apply(lambda x: "NG" if x == "NG-BIOMETHANE" else x)
        self.train.Ft = self.train.Ft.apply(
            lambda x: "ELECTRIC/HYDROGEN" if x == "HYDROGEN" or x == "ELECTRIC" else x)
        self.train.Ft = self.train.Ft.apply(
            lambda x: "HYBRID" if x == "PETROL/ELECTRIC" or x == "DIESEL/ELECTRIC" else x)
        self.train.Ft = self.train.Ft.apply(lambda x: "BIOCARB" if x == "NG" or x == "E85" or x == "LPG" else x)

        self.test.Ft = self.test.Ft.apply(lambda x: "PETROL" if x == "UNKNOWN" else x)
        self.test.Ft = self.test.Ft.apply(lambda x: "NG" if x == "NG-BIOMETHANE" else x)
        self.test.Ft = self.test.Ft.apply(lambda x: "ELECTRIC/HYDROGEN" if x == "HYDROGEN" or x == "ELECTRIC" else x)
        self.test.Ft = self.test.Ft.apply(
            lambda x: "HYBRID" if x == "PETROL/ELECTRIC" or x == "DIESEL/ELECTRIC" else x)
        self.test.Ft = self.test.Ft.apply(lambda x: "BIOCARB" if x == "NG" or x == "E85" or x == "LPG" else x)

        print("Valeurs manquantes catégorielles imputées ✅")


    def prepare_data(self):
        self.remove_train_nan()
        self.remove_test_nan()
        self.rename_columns()
        self.get_type_list()
        self.impute_train_test_numerical()
        self.impute_train_test_categorical()

        return self.train, self.test



# Import des données 

In [4]:
#df_train = pd.read_csv('/kaggle/input/estimate-co2-emissions-from-cars/train.csv')
#df_test = pd.read_csv('/kaggle/input/estimate-co2-emissions-from-cars/test.csv')


df_train = pd.read_csv('../data/train.csv')
df_test = pd.read_csv('../data/test.csv')

/var/folders/12/33kq22951gn7jfpz3d32v_qc0000gn/T/ipykernel_3800/358514020.py:5: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('../data/train.csv')
/var/folders/12/33kq22951gn7jfpz3d32v_qc0000gn/T/ipykernel_3800/358514020.py:6: DtypeWarning: Columns (27) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test = pd.read_csv('../data/test.csv')


# Préparation des données

In [5]:
dataprep = DataPreparation(df_train, df_test)
clean_train, clean_test = train_clean, test_clean = dataprep.prepare_data()

Valeurs manquantes du train supprimées ✅
Valeurs manquantes du test supprimées ✅
Variables renommées ✅
Valeurs manquantes numériques imputées ✅
Valeurs manquantes catégorielles imputées ✅


In [11]:
clean_train_avant_encoding = clean_train.copy()

# One Hot Encoding

In [6]:
ft_column = clean_train['Ft']

# Appliquez l'encodage one-hot à la colonne 'Ft' avec un préfixe
encoded_ft = pd.get_dummies(ft_column, prefix='Ft', dtype=int)

# Ajoutez les colonnes encodées à votre DataFrame
clean_train = pd.concat([clean_train, encoded_ft], axis=1)

# Supprimez la colonne originale 'Ft' si vous le souhaitez
clean_train = clean_train.drop(columns=['Ft'])

ft_column = clean_train['Fm']

# Appliquez l'encodage one-hot à la colonne 'Ft' avec un préfixe
encoded_ft = pd.get_dummies(ft_column, prefix='Fm', dtype=int)

# Ajoutez les colonnes encodées à votre DataFrame
clean_train = pd.concat([clean_train, encoded_ft], axis=1)

# Supprimez la colonne originale 'Ft' si vous le souhaitez
clean_train = clean_train.drop(columns=['Fm'])


# -----------------------------------------------------Pareil pour test -----------------------------------------------------

ft_column = clean_test['Ft']

# Appliquez l'encodage one-hot à la colonne 'Ft' avec un préfixe
encoded_ft = pd.get_dummies(ft_column, prefix='Ft', dtype=int)

# Ajoutez les colonnes encodées à votre DataFrame
clean_test = pd.concat([clean_test, encoded_ft], axis=1)

# Supprimez la colonne originale 'Ft' si vous le souhaitez
clean_test = clean_test.drop(columns=['Ft'])

ft_column = clean_test['Fm']

# Appliquez l'encodage one-hot à la colonne 'Ft' avec un préfixe
encoded_ft = pd.get_dummies(ft_column, prefix='Fm', dtype=int)

# Ajoutez les colonnes encodées à votre DataFrame
clean_test = pd.concat([clean_test, encoded_ft], axis=1)

# Supprimez la colonne originale 'Ft' si vous le souhaitez
clean_test = clean_test.drop(columns=['Fm'])

# Impact Encoding

In [7]:
# Imaginons que `clean_train` est votre jeu de données d'entraînement et `clean_test` est votre jeu de données de test

# Initialiser un dictionnaire pour stocker les dictionnaires d'impact de chaque caractéristique catégorielle
impact_dicts = {}

for categorical_feature in ['T', 'Man', 'VFN', 'Tan', 'Cn', 'Mk', 'Va']:
    # Calculer la moyenne de la valeur cible pour chaque catégorie
    category_means = clean_train.groupby(categorical_feature)['Ewltp_(g/km)'].mean()

    # Calculer la différence de la moyenne de la valeur cible entre chaque catégorie et la moyenne globale
    category_impacts = category_means - category_means.mean()

    # Stocker l'impact de la catégorie dans un dictionnaire
    impact_dicts[categorical_feature] = category_impacts.to_dict()

    # Encoder la caractéristique catégorielle dans le jeu de données d'entraînement
    clean_train['encoded_' + categorical_feature] = clean_train[categorical_feature].map(impact_dicts[categorical_feature])

    # Supprimer la caractéristique catégorielle originale du jeu de données d'entraînement
    clean_train.drop(columns=categorical_feature, inplace=True)

# Appliquer les mêmes encodages aux données de test
for categorical_feature in ['T', 'Man', 'VFN', 'Tan', 'Cn', 'Mk', 'Va']:
    # Si une valeur dans le jeu de données de test n'a pas été vue dans le jeu de données d'entraînement,
    # on peut décider de la remplacer par 0 ou la moyenne globale des impacts calculés sur le jeu de données d'entraînement
    default_impact = impact_dicts[categorical_feature].get("Default", 0)

    # Encoder la caractéristique catégorielle dans le jeu de données de test
    clean_test['encoded_' + categorical_feature] = clean_test[categorical_feature].apply(lambda x: impact_dicts[categorical_feature].get(x, default_impact))

    # Supprimer la caractéristique catégorielle originale du jeu de données de test
    clean_test.drop(columns=categorical_feature, inplace=True)


In [11]:
clean_train.columns

Index(['ID', 'Country', 'Mh', 'Ve', 'Ct', 'Cr', 'm_(kg)', 'Mt', 'Ewltp_(g/km)',
       'W_(mm)', 'At1_(mm)', 'At2_(mm)', 'ec_(cm3)', 'ep_(KW)',
       'Fuel_consumption_', 'Electric_range_(km)', 'Ft_BIOCARB', 'Ft_DIESEL',
       'Ft_ELECTRIC/HYDROGEN', 'Ft_HYBRID', 'Ft_PETROL', 'Fm_B', 'Fm_E',
       'Fm_F', 'Fm_H', 'Fm_M', 'Fm_P', 'encoded_T', 'encoded_Man',
       'encoded_VFN', 'encoded_Tan', 'encoded_Cn', 'encoded_Mk', 'encoded_Va'],
      dtype='object')

In [12]:
clean_test.columns

Index(['ID', 'Country', 'Mh', 'Ve', 'Ct', 'Cr', 'm_(kg)', 'Mt', 'W_(mm)',
       'At1_(mm)', 'At2_(mm)', 'ec_(cm3)', 'ep_(KW)', 'Fuel_consumption_',
       'Electric_range_(km)', 'Ft_BIOCARB', 'Ft_DIESEL',
       'Ft_ELECTRIC/HYDROGEN', 'Ft_HYBRID', 'Ft_PETROL', 'Fm_B', 'Fm_E',
       'Fm_F', 'Fm_H', 'Fm_M', 'Fm_P', 'encoded_T', 'encoded_Man',
       'encoded_VFN', 'encoded_Tan', 'encoded_Cn', 'encoded_Mk', 'encoded_Va'],
      dtype='object')

# Standardization

In [13]:
from sklearn.preprocessing import StandardScaler


# Create a StandardScaler object
scaler = StandardScaler()

# Apply the StandardScaler to the data
data_scaled = scaler.fit_transform(clean_train.select_dtypes(include = 'float').drop(columns ='Ewltp_(g/km)'))
test_scaled = scaler.transform(clean_test.select_dtypes(include = 'float'))

# Save the scaled data
data_scaled = pd.DataFrame(data_scaled, columns=clean_train.select_dtypes(include = 'float').drop(columns ='Ewltp_(g/km)').columns)
clean_train_scaled = pd.concat([data_scaled,clean_train.select_dtypes(exclude = 'float')], axis = 1)

# Same with test
test_scaled = pd.DataFrame(test_scaled, columns=clean_test.select_dtypes(include = 'float').columns)
clean_test_scaled = pd.concat([test_scaled,clean_test.select_dtypes(exclude = 'float')], axis = 1)


# Train test split

In [14]:
X = clean_train_scaled.select_dtypes(exclude = 'object')
y = clean_train['Ewltp_(g/km)']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# XGBoost

In [15]:
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import KFold

N_ESTIMATORS = 100
MAX_DEPTH = 20
N_ITERATION = 100

# Créez un modèle XGBoost avec les mêmes paramètres
model_xgboost = xgb.XGBRegressor(n_estimators=N_ESTIMATORS, max_depth=MAX_DEPTH, random_state=10)

print("lancement !")

# Créez un objet KFold en spécifiant le nombre de plis (n_splits)
n_splits = 2  # Vous pouvez ajuster le nombre de plis selon vos besoins
kfold = KFold(n_splits=n_splits)

# Initialisez une liste pour stocker les scores de MAE
mae_scores = []

# Effectuez la validation croisée
for train_index, test_index in kfold.split(X_train):
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]

    # Fit en cours
    print("fit en cours... ⏳")
    model_xgboost.fit(X_train_fold, y_train_fold)
    print("fit terminé 🎉")

    # Predict en cours
    print("predict en cours... ⏳")
    y_pred_fold = model_xgboost.predict(X_test_fold)
    print("predict effectué 🎉")

    # Calcul de la MAE pour ce pli
    mae_fold = mean_absolute_error(y_test_fold, y_pred_fold)
    mae_scores.append(mae_fold)
    print(f"MAE {mae_fold}")

# Calcul de la moyenne des scores MAE de tous les plis
mae_mean = np.mean(mae_scores)
print(f"MAE moyenne de XGBoost avec {n_splits} plis : {mae_mean}")


lancement !
fit en cours... ⏳
fit terminé 🎉
predict en cours... ⏳
predict effectué 🎉
MAE 2.8705692969797303
fit en cours... ⏳
fit terminé 🎉
predict en cours... ⏳
predict effectué 🎉
MAE 2.867204048031602
MAE moyenne de XGBoost avec 2 plis : 2.868886672505666


# Predict

In [19]:
prediction = model_xgboost.predict(clean_test_scaled.select_dtypes(exclude = 'object'))

submission = clean_test_scaled[["ID"]].copy()  
submission["Ewltp (g/km)"] = prediction

In [22]:
submission.to_csv("../data/sample_submission_xgb.csv", index=False)